## Install the geocoder

In [1]:
!pip install geocoder


## Get the survey data

- [survey](https://docs.google.com/forms/d/1oF7us4ASuFpF6iOhn4g5MnXf4Z8E5EPaIe2mMdPw_cM/edit)
- [data](https://docs.google.com/spreadsheets/d/1qPtvXZJ-IoNXdbVcPxr4buVLUONGsYS7-xNGqPFP144/edit#gid=696092516)


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSg8WVXVb47Tri6-JLpYZ-rxXUr3iX5HRD23rzK5XmABGpQI3jfSZk9btVTiguQZIgIWAmecSjbXvUM/pub?gid=696092516&single=true&output=csv')
                 

In [4]:
df

,Timestamp,Where do you live? Enter a zipcode.,"If it were up to you, how would you want to work moving forward?"


## Rename columns

In [5]:
df.columns=['timestamp','zip','work']

In [6]:
df

,timestamp,zip,work


## Geocode the zipcodes

In [7]:
import geocoder

In [8]:
df['lat'] = ''
df['lon'] = ''

In [9]:
for index, row in df.iterrows():
    g = geocoder.mapbox(row.zip,key='pk.eyJ1IjoiY2h3b2lwcm9qZWN0IiwiYSI6ImNrdTRjZXkwaTRwaWwycXBtY290MjA5d3gifQ.iyENw-MBBY3_9tMPv7frFg')
    print(g.latlng)
    df.loc[index,'lat'] = g.latlng[0]
    df.loc[index,'lon'] = g.latlng[1]
    

In [10]:
df

,timestamp,zip,work,lat,lon


### Get average lat/lon's

In [11]:
# average latitude
latitude = df.lat.mean()
latitude

nan

In [12]:
# average longitude
longitude = df.lon.mean()
longitude

nan

## Map it!

In [13]:
import folium

In [14]:
m = folium.Map(location = [34.0703726,-118.4453799], tiles="Stamen Terrain")
# add a 10 km radius to ucla
folium.Circle(
    radius=20000,
    location=[34.0703726,-118.4453799],
    popup="20 km radius",
    tooltip="20 km radius",
    color="crimson",
    fill=True,
).add_to(m)

folium.Circle(
    radius=10000,
    location=[34.0703726,-118.4453799],
    popup="10 km radius",
    tooltip="10 km radius",
    color="crimson",
    fill=True,
).add_to(m)

for index, row in df.iterrows():
    tooltip = 'I live in zipcode '+str(row.zip)+' and want to work '+row.work
    # color
    if row.work == '100% remote':
        color = 'red'
    elif row.work == 'hybrid':
        color = 'darkblue'
    else:
        color = 'green'
    folium.Marker([row.lat, row.lon], 
                  popup=row.work, 
                  tooltip=tooltip,
                  icon=folium.Icon(color=color,icon='cloud'),
                 ).add_to(m)
m

In [16]:
# save the interactive maps as an html file
m.save('index.html')